In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('events_bu.csv')

In [2]:
df=pd.read_csv('processed_data.csv')
df

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,year,month,day,hour,main_category,sub_category,sub_sub_category
0,0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,2020,9,24,11,electronics,telephone,NaN
1,1,2020-09-24 11:57:26+00:00,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,2020,9,24,11,computers,components,cooler
2,2,2020-09-24 11:57:27+00:00,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,2020,9,24,11,NaN,NaN,NaN
3,3,2020-09-24 11:57:33+00:00,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,2020,9,24,11,computers,peripherals,printer
4,4,2020-09-24 11:57:36+00:00,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,2020,9,24,11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885124,885124,2021-02-28 23:55:01+00:00,view,953226,2144415927553229037,NaN,NaN,219.94,1515915625611023730,FRLqIttxKU,2021,2,28,23,NaN,NaN,NaN
885125,885125,2021-02-28 23:58:05+00:00,view,1715907,2144415927049912542,electronics.video.tv,starwind,80.03,1515915625611024014,g6WqPf50Ma,2021,2,28,23,electronics,video,tv
885126,885126,2021-02-28 23:58:09+00:00,view,4170534,2144415939364389423,electronics.clocks,amazfit,64.92,1515915625611024020,xNIJBqZdkd,2021,2,28,23,electronics,clocks,NaN
885127,885127,2021-02-28 23:58:14+00:00,view,888273,2144415921932861531,electronics.telephone,NaN,10.16,1515915625611024030,9pCbKMIcSx,2021,2,28,23,electronics,telephone,NaN


In [4]:
# 날짜별 집계를 위한 설정
df['event_date']=pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('event_date')

In [5]:
import pandas as pd

# event_time을 datetime으로 변환하고 event_date 열 추가
df['event_date'] = pd.to_datetime(df['event_time']).dt.date

# 기준 날짜 범위 생성
date_range = pd.date_range(start='2020-09-24', end='2021-02-28').date

# 결과를 저장할 딕셔너리 생성
retention_dict = {}
repeat_purchase_dict = {}

# 각 날짜에 대해 리텐션 계산
for base_date in date_range:
    # 기준 날짜부터 7일 동안의 데이터 필터링
    next_7_days = pd.date_range(start=base_date, periods=8).date
    filtered_data = df[df['event_date'].isin(next_7_days)]
    
    # 기준 날짜에 방문한 사용자 추출
    initial_users = filtered_data[filtered_data['event_date'] == base_date]['user_id'].unique()
    
    # 7일 이내 재방문 리텐션 계산
    if len(initial_users) > 0:
        retained_users_within_7_days = filtered_data[(filtered_data['event_date'] > base_date) & 
                                                     (filtered_data['event_date'] <= base_date + pd.Timedelta(days=7)) &
                                                     (filtered_data['user_id'].isin(initial_users))]['user_id'].nunique()
        retention_rate_within_7_days = round((retained_users_within_7_days / len(initial_users)) * 100, 3)
    else:
        retention_rate_within_7_days = 0.0
    
    # 7일 이내 재구매 리텐션 계산
    if len(initial_users) > 0:
        repeat_purchase_users_within_7_days = filtered_data[(filtered_data['event_date'] > base_date) & 
                                                            (filtered_data['event_date'] <= base_date + pd.Timedelta(days=7)) &
                                                            (filtered_data['event_type'] == 'purchase') &
                                                            (filtered_data['user_id'].isin(initial_users))]['user_id'].nunique()
        repeat_purchase_rate_within_7_days = round((repeat_purchase_users_within_7_days / len(initial_users)) * 100, 3)
    else:
        repeat_purchase_rate_within_7_days = 0.0
    
    # 결과 저장
    retention_dict[base_date] = retention_rate_within_7_days
    repeat_purchase_dict[base_date] = repeat_purchase_rate_within_7_days

# 딕셔너리를 시리즈로 변환
retention_series = pd.Series(retention_dict, name="7-Day Retention Rate (%)")
repeat_purchase_series = pd.Series(repeat_purchase_dict, name="7-Day Repeat Purchase Rate (%)")

# 결과 출력
print(retention_series)
print(repeat_purchase_series)


2020-09-24    8.542
2020-09-25    9.996
2020-09-26    8.473
2020-09-27    9.511
2020-09-28    9.187
              ...  
2021-02-24    8.815
2021-02-25    7.543
2021-02-26    6.081
2021-02-27    5.353
2021-02-28    0.000
Name: 7-Day Retention Rate (%), Length: 158, dtype: float64
2020-09-24    0.884
2020-09-25    0.653
2020-09-26    0.591
2020-09-27    0.823
2020-09-28    1.025
              ...  
2021-02-24    0.792
2021-02-25    0.754
2021-02-26    0.743
2021-02-27    0.465
2021-02-28    0.000
Name: 7-Day Repeat Purchase Rate (%), Length: 158, dtype: float64


In [6]:
df['event_time'] = pd.to_datetime(df['event_time']).dt.tz_localize(None)
df['date'] = df['event_time'].dt.date

daily_event_counts = df.groupby(['date', 'event_type']).size().unstack(fill_value=0)

daily_event_counts['vc_rate'] = daily_event_counts['cart'] / daily_event_counts['view']
daily_event_counts['cp_rate'] = daily_event_counts['purchase'] / daily_event_counts['cart']

daily_event_counts.fillna(0, inplace=True)

print(daily_event_counts[['vc_rate', 'cp_rate']])

event_type   vc_rate   cp_rate
date                          
2020-09-24  0.047550  0.848485
2020-09-25  0.062305  0.620833
2020-09-26  0.055854  0.672316
2020-09-27  0.048193  0.639535
2020-09-28  0.060870  0.772201
...              ...       ...
2021-02-24  0.081012  0.707022
2021-02-25  0.082398  0.603406
2021-02-26  0.080418  0.661058
2021-02-27  0.074116  0.686154
2021-02-28  0.071610  0.575221

[158 rows x 2 columns]


In [7]:
for name,groups in gr:
    print(name)
    print(groups.head())

2020-09-24
           event_time event_type  product_id          category_id  \
0 2020-09-24 11:57:06       view     1996170  2144415922528452715   
1 2020-09-24 11:57:26       view      139905  2144415926932472027   
2 2020-09-24 11:57:27       view      215454  2144415927158964449   
3 2020-09-24 11:57:33       view      635807  2144415923107266682   
4 2020-09-24 11:57:36       view     3658723  2144415921169498184   

                   category_code        brand   price              user_id  \
0          electronics.telephone          NaN   31.90  1515915625519388267   
1    computers.components.cooler       zalman   17.16  1515915625519380411   
2                            NaN          NaN    9.81  1515915625513238515   
3  computers.peripherals.printer       pantum  113.81  1515915625519014356   
4                            NaN  cameronsino   15.87  1515915625510743344   

  user_session  year  month  day  hour main_category sub_category  \
0   LJuJVLEjPT  2020      9   24    

In [8]:
unique_view_sessions=gr.apply(lambda x: x.loc[x['event_type']=='view','user_session'].nunique()) #1
unique_view_sessions

event_date
2020-09-24    1440
2020-09-25    2627
2020-09-26    2157
2020-09-27    2340
2020-09-28    2902
              ... 
2021-02-24    3239
2021-02-25    3095
2021-02-26    3256
2021-02-27    2834
2021-02-28    3042
Length: 158, dtype: int64

In [9]:
from functools import reduce

df['event_date']=pd.to_datetime(df['event_time']).dt.date
gr=df.groupby('event_date')

# Acqusition
DAU = df.groupby('event_date').agg(DAU=('user_id', lambda x: x.nunique()))
DAS = df.groupby('event_date').agg(DAS=('user_session', lambda x: x.nunique()))

Acqusition = pd.merge(DAS, DAU, on='event_date', how ='inner').reset_index()

# Activation
Activation = gr.agg(
    
    sessions_per_user=('user_session', lambda x: (x.count() / gr.get_group(x.name)['user_id'].nunique()).round(2)),
    view_count=('event_type', lambda x: (x == 'view').sum()),
    cart_count=('event_type', lambda x: (x == 'cart').sum()),
    purchase_count=('event_type', lambda x: (x == 'purchase').sum()),
    unique_view_sessions = ('user_session',lambda x: x[df['event_type']=='view'].nunique()), 
    unique_cart_sessions = ('user_session',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_sessions = ('user_session',lambda x: x[df['event_type']=='purchase'].nunique()),
    unique_view_users = ('user_id',lambda x: x[df['event_type']=='view'].nunique()),
    unique_cart_users = ('user_id',lambda x: x[df['event_type']=='cart'].nunique()),
    unique_purchase_users = ('user_id',lambda x: x[df['event_type']=='purchase'].nunique())
    
                    ).reset_index()

# Revenue
Revenue = gr.agg(
    
    total_purchase_amount = ('price', lambda x: (x[df['event_type']=='purchase'].sum()).round(2)),
    purchase_conversion_rate = ('event_type',lambda x: (x=='purchase').sum()/
                                gr.get_group(x.name)['user_id'].nunique()), 
    ARPU = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_id'].nunique()).round(2)), 
    ARPS = ('price',lambda x: (x[df['event_type']=='purchase'].sum()/
                               gr.get_group(x.name)['user_session'].nunique()).round(2)),
    high_brand_daily = ('price', lambda x: x[df['event_type']=='purchase'].groupby(df['brand']).sum().idxmax()),
    AOV=('price', lambda x: ((x[df['event_type'] == 'purchase'].sum())/
                         (x[df['event_type'] == 'purchase'].count())).round(2))
                            
                ).reset_index()

merge_df = reduce(lambda x,y: pd.merge(x,y, on='event_date', how='inner'), [Acqusition, Activation, Revenue])
merge_df.head()

,event_date,DAS,DAU,sessions_per_user,view_count,cart_count,purchase_count,unique_view_sessions,unique_cart_sessions,unique_purchase_sessions,unique_view_users,unique_cart_users,unique_purchase_users,total_purchase_amount,purchase_conversion_rate,ARPU,ARPS,high_brand_daily,AOV
0,2020-09-24,1446,1358,1.67,2082,99,84,1440,83,58,1355,81,56,9563.67,0.061856,7.04,6.61,asus,113.85
1,2020-09-25,2637,2451,1.73,3852,240,149,2627,199,109,2445,193,105,13025.72,0.060792,5.31,4.94,amd,87.42
2,2020-09-26,2164,2030,1.71,3169,177,119,2157,132,76,2028,132,73,11821.97,0.058621,5.82,5.46,epson,99.34
3,2020-09-27,2349,2187,1.76,3569,172,110,2340,142,72,2183,137,70,7910.53,0.050297,3.62,3.37,gigabyte,71.91
4,2020-09-28,2913,2732,1.73,4255,259,200,2902,201,123,2726,197,118,15120.82,0.073206,5.53,5.19,epson,75.60


In [10]:
# (리텐션/1,3,7,30)

# 'event_time'을 datetime 형식으로 변환하고 'event_date' 열 생성
df['event_time'] = pd.to_datetime(df['event_time']).dt.tz_localize(None)
df['event_date'] = df['event_time'].dt.date

# 기준 날짜 범위 설정
start_date = pd.to_datetime('2020-09-24')
end_date = pd.to_datetime('2021-02-28')

# 결과를 저장할 리스트 생성
retention_results = []

# 각 날짜에 대해 리텐션 계산
for base_date in pd.date_range(start=start_date, end=end_date):
    base_date = base_date.date()
    next_30_days = pd.date_range(start=base_date, periods=31).date
    filtered_data = df[df['event_date'].isin(next_30_days)]

    # 기준 날짜에 방문한 사용자 추출
    initial_users = filtered_data[filtered_data['event_date'] == base_date]['user_id'].unique()

    # 각 기간별 재방문 리텐션 계산
    for days in [1, 3, 7, 30]:
        if len(initial_users) > 0:
            retained_users = filtered_data[(filtered_data['event_date'] > base_date) &
                                           (filtered_data['event_date'] <= base_date + pd.Timedelta(days=days)) &
                                           (filtered_data['user_id'].isin(initial_users))]['user_id'].nunique()
            retention_rate = round((retained_users / len(initial_users)) * 100, 3)
        else:
            retention_rate = 0.0

        # 결과 저장
        retention_results.append({
            'base_date': base_date,
            'days': f'{days}-Day Retention Rate (%)',
            'retention_rate': retention_rate
        })

# ---------------------------------------------------------------------------------------------------------------------------
purchase_retention_results = []

# 각 날짜에 대해 리텐션 계산
for base_date in pd.date_range(start=start_date, end=end_date):
    base_date = base_date.date()
    next_30_days = pd.date_range(start=base_date, periods=31).date
    filtered_data = df[df['event_date'].isin(next_30_days)]
    
    # 기준 날짜에 방문한 사용자 추출
    initial_users = filtered_data[filtered_data['event_date'] == base_date]['user_id'].unique()
    
    # 각 기간별 구매 리텐션 계산
    for days in [1, 3, 7, 30]:
        if len(initial_users) > 0:
            purchase_users = filtered_data[(filtered_data['event_date'] > base_date) & 
                                           (filtered_data['event_date'] <= base_date + pd.Timedelta(days=days)) &
                                           (filtered_data['event_type'] == 'purchase') &
                                           (filtered_data['user_id'].isin(initial_users))]['user_id'].nunique()
            purchase_retention_rate = round((purchase_users / len(initial_users)) * 100, 3)
        else:
            purchase_retention_rate = 0.0
        
        # 결과 저장
        purchase_retention_results.append({
            'base_date': base_date,
            'days': f'{days}-Day Purchase Retention Rate (%)',
            'purchase_retention_rate': purchase_retention_rate
        })


# ----------------------------------------------------------------------------------------------------------------------------------------
retention_df = pd.DataFrame(retention_results)
retention_df_pivot = retention_df.pivot(index='base_date', columns='days', values='retention_rate')
Retention = retention_df_pivot[['1-Day Retention Rate (%)', '3-Day Retention Rate (%)',
                                   '7-Day Retention Rate (%)', '30-Day Retention Rate (%)']]

Retention = Retention.reset_index().rename_axis(None, axis=1)
Retention.rename(columns={'base_date': 'event_date'}, inplace=True)

# ------------------------------------------------------------------------------------------------------------------------------

purchase_retention_df = pd.DataFrame(purchase_retention_results)
purchase_retention_pivot = purchase_retention_df.pivot(index='base_date', columns='days', values='purchase_retention_rate')
purchase_retention_df = purchase_retention_pivot[['1-Day Purchase Retention Rate (%)', '3-Day Purchase Retention Rate (%)',
                                                  '7-Day Purchase Retention Rate (%)', '30-Day Purchase Retention Rate (%)']]


purchase_retention_df = purchase_retention_df.reset_index().rename_axis(None, axis=1)
purchase_retention_df.rename(columns={'base_date': 'event_date'}, inplace=True)


In [11]:
Retention.head()

,event_date,1-Day Retention Rate (%),3-Day Retention Rate (%),7-Day Retention Rate (%),30-Day Retention Rate (%)
0,2020-09-24,5.007,6.701,8.542,11.193
1,2020-09-25,4.325,7.589,9.996,12.444
2,2020-09-26,4.828,6.995,8.473,10.936
3,2020-09-27,5.350,7.956,9.511,12.071
4,2020-09-28,5.490,7.833,9.187,11.567


In [12]:
purchase_retention_df.head()

,event_date,1-Day Purchase Retention Rate (%),3-Day Purchase Retention Rate (%),7-Day Purchase Retention Rate (%),30-Day Purchase Retention Rate (%)
0,2020-09-24,0.368,0.442,0.884,1.399
1,2020-09-25,0.122,0.245,0.653,1.224
2,2020-09-26,0.197,0.443,0.591,1.084
3,2020-09-27,0.274,0.503,0.823,1.417
4,2020-09-28,0.439,0.732,1.025,1.574


In [13]:
merge_ret=pd.merge(Retention, purchase_retention_df, on='event_date', how='inner')

In [14]:
merge_all = pd.merge(merge_ret,merge_df, on='event_date', how='inner')
merge_all.T

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,157
event_date,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,...,2021-02-19,2021-02-20,2021-02-21,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-02-27,2021-02-28
1-Day Retention Rate (%),5.007,4.325,4.828,5.35,5.49,5.434,5.048,5.372,4.455,4.718,...,5.169,5.242,6.013,5.661,5.123,5.579,5.496,4.324,5.353,0.0
3-Day Retention Rate (%),6.701,7.589,6.995,7.956,7.833,8.116,6.95,6.669,7.698,6.799,...,8.173,7.955,8.554,8.4,7.903,8.368,7.543,6.081,5.353,0.0
7-Day Retention Rate (%),8.542,9.996,8.473,9.511,9.187,9.527,8.851,9.337,9.066,8.696,...,9.955,10.264,10.236,9.971,9.492,8.815,7.543,6.081,5.353,0.0
30-Day Retention Rate (%),11.193,12.444,10.936,12.071,11.567,12.315,11.339,11.782,11.372,10.546,...,10.583,10.447,10.236,9.971,9.492,8.815,7.543,6.081,5.353,0.0
1-Day Purchase Retention Rate (%),0.368,0.122,0.197,0.274,0.439,0.318,0.293,0.185,0.352,0.416,...,0.733,0.733,0.68,0.365,0.635,0.344,0.503,0.507,0.465,0.0
3-Day Purchase Retention Rate (%),0.442,0.245,0.443,0.503,0.732,0.741,0.475,0.259,0.664,0.694,...,1.188,1.063,1.002,0.767,0.913,0.689,0.754,0.743,0.465,0.0
7-Day Purchase Retention Rate (%),0.884,0.653,0.591,0.823,1.025,0.988,0.695,0.741,1.055,0.925,...,1.502,1.32,1.288,0.986,1.033,0.792,0.754,0.743,0.465,0.0
30-Day Purchase Retention Rate (%),1.399,1.224,1.084,1.417,1.574,1.517,0.988,0.963,1.485,1.526,...,1.607,1.356,1.288,0.986,1.033,0.792,0.754,0.743,0.465,0.0
DAS,1446,2637,2164,2349,2913,3035,2913,2847,2728,2271,...,3187,3011,3139,3045,2805,3250,3113,3268,2846,3054


In [15]:
merge_all.to_csv('merge_all.csv')